# Data Analysis of the Features

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from scripts.data_preprocessing import set_nan, convert_nan, standardize_data
from scripts.proj1_helpers import load_csv_data

Loading of the dataset and setting all -999 values to nan

In [4]:
y, x, ids = load_csv_data("data/train.csv")

x = set_nan(x)

First, as noted in the description of the features, some of them are left undefined in specific circumstances. We proceeded to check if the database corresponds to this description by veryfing that the occurences of undefined values are all grouped as they should be. 

In [5]:
def count_nan(x):
    """
    Counts nan values in each feature, then outputs ratio of nan/total values. 
    
    :param x: input
    :return: vector composed of ratios of nan for each feature
    """
    x = set_nan(x)
    truth_array = np.isnan(x)
    return ((np.sum(truth_array, axis=0)) / x.shape[0])*100

In [6]:
def check_nan_positions(x, features):
    """
    Checks if the nans occur in all the selected features, outputs the percentage of values with nans 
    that are in all the chosen columns 
    
    :param x: input
    :return: value: percentage of nans present in all columns
    """

    c = np.isnan(x[:, features])
    check = np.sum(c, 1) == features.shape[0]

    value = np.sum(check) / check.shape[0]
    return value * 100

In [7]:
nan_ratios = count_nan(x) # vector of ratios of nan for each feature
print(nan_ratios)
features1 = np.array([4, 5, 6, 12, 26, 27, 28]) # features set undefined if the number of jets is <=1
value1 = check_nan_positions(x, features1)
print(value1)
features2 = np.array([23, 24, 25]) # features set undefined if number of jets = 0 
value2 = check_nan_positions(x, features2)
print(value2)
features3 = np.array([0, 23, 24, 25, 4, 5, 6, 12, 26, 27, 28]) 
# Counts how may datapoints have n_jets <=1 and, at the same time, 
# have an undefined theorical mass of the boson (feature 0)
value3 = check_nan_positions(x, features3)
print(value3)

[15.2456  0.      0.      0.     70.9828 70.9828 70.9828  0.      0.
  0.      0.      0.     70.9828  0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.     39.9652 39.9652 39.9652 70.9828
 70.9828 70.9828  0.    ]
70.9828
39.9652
10.4492


Then we verified how balanced the dataset is, and how balanced are the subsets characterized by a specific jet number, that have nans in the features outlined above.

In [8]:
def cut_datapoints(y, x, features):
    """
    Eliminates datapoints which have nans in all features of a specific set 
    
    :param x: input datapoints
    :param y: input event or background
    :param features: features in which nans are present
    :return: y,x
    
    """

    c = np.isnan(x[:, features])
    check = np.sum(c, 1) == features.shape[0]
    check = np.logical_not(check)
    x = x[check, :]
    y = y[check]
    return y, x

In [9]:
proportion_hits = np.sum(y[y == 1]) / y.shape[0] # Entire dataset
print(proportion_hits)
# Cutting of the datasets 
features4 = np.array([23, 24, 25, 4, 5, 6, 12, 26, 27, 28])
y1, x1 = cut_datapoints(y, x, features1) # cutting jet numbers 0 and 1
y2, x2 = cut_datapoints(y, x, features2) # cutting jet number 0

0.342668


Calculation of the portion of signal in the segments remaining after the cut

In [10]:
proportion_hits1 = np.sum(y1[y1 == 1]) / y1.shape[0]
print(proportion_hits1) # 29% remaining with number of jets higher than 1
proportion_hits2 = np.sum(y2[y2 == 1]) / y2.shape[0]
print(proportion_hits2) 

0.4475276732420771
0.40093412487423963


Are there features that are a linear combination of other features? This function checks and outputs the indices of features that can be built by using a linear combination of others.

In [11]:
def linear_dependancy(x):
    # define the matrix containing the inner products of the columns
    inn_prod = x.T @ x
    # define the matrix containing the products of the norms of the columns
    arr_norm = np.linalg.norm(x, axis=0)[..., np.newaxis]
    norm_prod = arr_norm @ arr_norm.T
    # define the difference matrix
    diff = inn_prod - norm_prod
    # define indices where the difference is = 0
    # the indices represents the linearly dependent columns
    ind_dep = []
    for i in range(diff.shape[0]):
        for j in range(diff.shape[0]):
            if i != j:
                if np.abs(diff[i, j]) < 1E-1:
                    id = np.array([i, j])
                    ind_dep.append(id)
    return ind_dep

In [12]:
linear_dependancy(x)
o= np.array([[1,2],[3,6],[3,8]])
print(o)
linear_dependancy(o)

[[1 2]
 [3 6]
 [3 8]]


[]

To find further relationships between variables, the covariance matrix was computed.

In [29]:
def covariance_matrix(x):
    """
    Computes the covariance matrix computed with pearson's coefficient.
    :param x: input 
    :return: covmat : covariance matrix
    """
    xm = convert_nan(x, mode='mean')
    xm = standardize_data(x) # Standardizes and substitutes nans with the mean of values of that feature
    covmat = np.corrcoef(xm.T)
    return covmat

The computed covariance values are then selected using a threshold with the two helper functions below, that output the correlated pairs of features.

In [ ]:
covariance = covariance_matrix(x) 

def set_cov_inf(cov):
    """Sets the bottom half of the cov matrix as 0 to avoid repetition of related couples"""
    cov_ = cov
    for i in range(cov.shape[0]):
        for j in range(cov.shape[1]):
            if i >= j:
                cov_[i, j] = 0
    return cov_


def corr_col(cov, threshold):
    """initializes an array filled with the columns to be eliminated"""
    # threshold is the threshold correlation
    cde = []
    for i in range(cov.shape[0]):
        for j in range(cov.shape[1]):
            if abs(cov[i, j]) > threshold :
                v = np.array([i, j])
                cde.append(v)
    return cde

cov_ = set_cov_inf(covariance)
col_el = corr_col(cov_, threshold=0.9)
print(col_el, '\n')

cov_ = set_cov_inf(covariance)
col_el = corr_col(cov_, threshold=0.8)
print(col_el,'\n')

cov_ = set_cov_inf(covariance)
col_el = corr_col(cov_, threshold=0.6)
print(col_el,'\n')


Then we drew the histograms of the distributions of the features, excluding nan values,
overlapping signal and background distributions to highlight potential differences.

In [ ]:
y, x, ids = load_csv_data("data/train.csv")

x = set_nan(x)

figure1 = plt.figure(1)
for i in range(30):
    plt.subplot(5, 6, i + 1)
    k = x[:, i]
    kt = k[y == 1]
    kf = k[y == -1]

    plt.hist(kt[~np.isnan(kt)], bins='auto', alpha=0.5, facecolor='b')
    plt.hist(kf[~np.isnan(kf)], bins='auto', alpha=0.5, facecolor='r')
    plt.title(f'feature : {i}')
    plt.axis('tight')

It seems features 15, 18, 20, 25 and 28 do not add information, as the distribution of hits and misses are very similar and constant. To make sure that the ratio between hits and misses does not change depending on the value of the feature, we drew lineplots of the ratios.
if this ratio is constant then the feature does not have predictive power.

In [ ]:
figure2 = plt.figure(2)
L = 50  # number of bins
ratio = np.zeros(L)
for i in range(30):
    plt.subplot(5, 6, i + 1)
    k = x[:, i]
    kt = k[y == 1]
    kf = k[y == -1]
    kt = kt[~np.isnan(kt)]
    kf = kf[~np.isnan(kf)]
    kthist = np.histogram(kt, bins=L, range=(np.min(k[~np.isnan(k)]), np.max(k[~np.isnan(k)])))
    kfhist = np.histogram(kf, bins=L, range=(np.min(k[~np.isnan(k)]), np.max(k[~np.isnan(k)])))
    for j in range(L):
        if kfhist[0][j] == 0 or kthist[0][j] == 0:
            ratio[j] = 0
        else:
            ratio[j] = kthist[0][j] / kfhist[0][j]
    binz = kthist[1][0:L]
    plt.plot(binz, ratio)
    plt.title(f'feature : {i}')
    plt.ylim(0, 2)